In [1]:
!date

Sat Feb 15 11:26:36 UTC 2020


# リソース構築

In [2]:
import json
import datetime

# VPC
vpc_name="topse-vpc"
subnet_name="topse-subnet"
region="us-west1"
ip_range="10.1.0.0/20"

# GKE
cluster_name = "topse-cluster"
zones = ["us-west1-a","us-west1-b"]

# VPC Peering
vpc_peering_name="topse-vpc-peering"

# Cloud SQL
#  need to change instance name uniquely even if instance is deleted. The instance name is remaining for a while
suffix = format(datetime.datetime.now(), '%Y%m%d%H%M%S')
cloud_sql_name = "topse-wordpress-mysql-" + suffix
database_version = "MYSQL_5_7"
root_password = "8sg3jJguHLq8KjNk"
cloud_sql_ip=""

# Cloud DNS
private_zone_name="topse-private-zone"
dns_name="topse.private."

# FileStore
filestore_name="nfs-server"
volume_capacity="1TB"
filestore_ip=""

## VPC作成

In [3]:
!gcloud compute networks create {vpc_name} --description="topse custom network" --subnet-mode=custom

Created [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/networks/topse-vpc].
NAME       SUBNET_MODE  BGP_ROUTING_MODE  IPV4_RANGE  GATEWAY_IPV4
topse-vpc  CUSTOM       REGIONAL

Instances on this network will not be reachable until firewall rules
are created. As an example, you can allow all internal traffic between
instances as well as SSH, RDP, and ICMP by running:

$ gcloud compute firewall-rules create <FIREWALL_NAME> --network topse-vpc --allow tcp,udp,icmp --source-ranges <IP_RANGE>
$ gcloud compute firewall-rules create <FIREWALL_NAME> --network topse-vpc --allow tcp:22,tcp:3389,icmp



## サブネット作成

In [4]:
!gcloud compute networks subnets create {subnet_name} --network={vpc_name} --region={region} --range={ip_range}

Created [https://www.googleapis.com/compute/v1/projects/topse-nagaku/regions/us-west1/subnetworks/topse-subnet].
NAME          REGION    NETWORK    RANGE
topse-subnet  us-west1  topse-vpc  10.1.0.0/20


## ファイアウォールルール作成

In [5]:
!gcloud compute firewall-rules create {vpc_name}-ssh-ingress --description="general firewall to allow ssh for all instances in {vpc_name}" \
--direction=INGRESS --priority=1000 --network={vpc_name} --action=ALLOW --rules=tcp:22
!gcloud compute firewall-rules create {vpc_name}-mysql-ingress --description="firewall to allow mysql for wordpress instances in {vpc_name}" \
--direction=INGRESS --priority=1000 --network={vpc_name} --action=ALLOW --rules=tcp:3306 --source-ranges={ip_range}

Creating firewall...⠧Created [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/firewalls/topse-vpc-ssh-ingress].
Creating firewall...done.                                                      
NAME                   NETWORK    DIRECTION  PRIORITY  ALLOW   DENY  DISABLED
topse-vpc-ssh-ingress  topse-vpc  INGRESS    1000      tcp:22        False
Creating firewall...⠧Created [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/firewalls/topse-vpc-mysql-ingress].
Creating firewall...done.                                                      
NAME                     NETWORK    DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
topse-vpc-mysql-ingress  topse-vpc  INGRESS    1000      tcp:3306        False


## VPC Peering作成

In [6]:
!gcloud compute addresses create {vpc_peering_name} \
    --global \
    --purpose=VPC_PEERING \
    --prefix-length=16 \
    --description="for VPC peering to gcp service" \
    --network={vpc_name}

Created [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/addresses/topse-vpc-peering].


以下の gcloud services vpc-peerings update がエラーになった場合、エラーメッセージの案内に従って、[APIとサービス] からCloud Resource Manager APIを有効にする。

In [7]:
!gcloud services vpc-peerings update \
    --service=servicenetworking.googleapis.com \
    --ranges={vpc_peering_name} --network={vpc_name} \
    --force --quiet

Operation "operations/pssn.e3de7fd5-b7ae-4a7d-99e1-37a1f0dcc8e6" finished successfully.


# GKEクラスタの構築(Kubernatesクラスタ)

In [8]:
!gcloud container clusters create {cluster_name} \
--region {region} \
--node-locations  {zones[0]},{zones[1]} \
--enable-ip-alias \
--network {vpc_name} \
--subnetwork {subnet_name} \
--num-nodes=1

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
Creating cluster topse-cluster in us-west1... Cluster is being health-checked..
.⠧                                                                             
Creating cluster topse-cluster in us-west1... Cluster is being health-checked (
master is healthy)...done.                                                     
Created [https://container.googleapis.com/v1/projects/topse-nagaku/zones/us-west1/clusters/topse-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-west1/topse-cluster?project=topse-nagaku
kubeconfig entry generated for topse-cluster.
NAME           LOCATION  MASTER_VERSION  MASTER_IP        MACHINE_TYPE   NODE_VERSION    NUM_NODES  STATUS
topse-cluster  us-west1  1.14.10-gke.17  104.199.123.238  n1-standard-1  1.14.10-gke.17  2        

## GKE認証情報設定

In [9]:
!gcloud container clusters get-credentials {cluster_name} --region {region}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for topse-cluster.


# FileStoreの構築(NFS)

## FileStore作成

In [10]:
!gcloud beta filestore instances create {filestore_name} --zone {zones[0]} \
--tier=STANDARD --file-share=name="vol1",capacity={volume_capacity} --network=name={vpc_name}

Waiting for [operation-1581766344571-59e9bad15fb17-d8e9c5ed-ce00cf28] to finish
...done.                                                                       


## IPアドレス取得

In [11]:
output=!gcloud beta filestore instances describe {filestore_name} --zone {zones[0]} --format json
import json
while output[0].find('{') == -1:
    del output[0]
output[0] = "{"
json_result = json.loads(''.join(output))
print(json_result["networks"][0]["ipAddresses"][0])
filestore_ip=json_result["networks"][0]["ipAddresses"][0]

10.196.216.218


# Cloud SQL構築(DB)

## Cloud SQL作成

作成時間4分

In [12]:
!gcloud beta sql instances create {cloud_sql_name} --no-assign-ip --network={vpc_name} --no-backup --database-version={database_version} \
--root-password={root_password} --region {region}

Creating Cloud SQL instance...done.                                            
Created [https://sqladmin.googleapis.com/sql/v1beta4/projects/topse-nagaku/instances/topse-wordpress-mysql-20200215112636].
NAME                                  DATABASE_VERSION  LOCATION    TIER              PRIMARY_ADDRESS  PRIVATE_ADDRESS  STATUS
topse-wordpress-mysql-20200215112636  MYSQL_5_7         us-west1-a  db-n1-standard-1  -                10.253.0.3       RUNNABLE


## IPアドレス取得

In [13]:
output=!gcloud beta sql instances describe {cloud_sql_name} --format json
import json
while output[0].find('{') == -1:
    del output[0]
output[0] = "{"
json_result = json.loads(''.join(output))
print(json_result["ipAddresses"][0]["ipAddress"])
cloud_sql_ip=json_result["ipAddresses"][0]["ipAddress"]

10.253.0.3


## Secret作成

MySQLのrootパスワードをKubernatesのSecretに登録する

In [14]:
!kubectl create secret generic mysql --from-literal=password={root_password}

secret/mysql created


# Cloud DNSの構築

## プライベートゾーン作成

In [15]:
!gcloud beta dns managed-zones create {private_zone_name} --dns-name={dns_name} \
--description={private_zone_name} --visibility=private --networks {vpc_name}

Created [https://dns.googleapis.com/dns/v1beta2/projects/topse-nagaku/managedZones/topse-private-zone].


## DNSレコード追加

Cloud SQLのIPアドレス、ホスト名を登録する。

In [16]:
#!gcloud dns record-sets transaction abort --zone={private_zone_name}
!gcloud dns record-sets transaction start --zone={private_zone_name}
!gcloud dns record-sets transaction add {cloud_sql_ip} --name=mysql.{dns_name} --ttl=300 --type=A --zone={private_zone_name}
!gcloud dns record-sets transaction execute --zone={private_zone_name}

Transaction started [transaction.yaml].
Record addition appended to transaction at [transaction.yaml].
Executed transaction [transaction.yaml] for managed-zone [topse-private-zone].
Created [https://dns.googleapis.com/dns/v1/projects/topse-nagaku/managedZones/topse-private-zone/changes/1].
ID  START_TIME                STATUS
1   2020-02-15T11:42:24.901Z  pending


FileStoreのIPアドレス、ホスト名を登録する。

In [17]:
#!gcloud dns record-sets transaction abort --zone={private_zone_name}
!gcloud dns record-sets transaction start --zone={private_zone_name}
!gcloud dns record-sets transaction add {filestore_ip} --name=nfs.{dns_name} --ttl=300 --type=A --zone={private_zone_name}
!gcloud dns record-sets transaction execute --zone={private_zone_name}

Transaction started [transaction.yaml].
Record addition appended to transaction at [transaction.yaml].
Executed transaction [transaction.yaml] for managed-zone [topse-private-zone].
Created [https://dns.googleapis.com/dns/v1/projects/topse-nagaku/managedZones/topse-private-zone/changes/2].
ID  START_TIME                STATUS
2   2020-02-15T11:42:29.845Z  pending


リソース削除用にパラメータを保存

In [18]:
res_info = {'vender': 'gcp',
            'cloud_sql_ip': cloud_sql_ip,
            'cloud_sql_name': cloud_sql_name,
            'cluster_name': cluster_name,
            'dns_name': dns_name,
            'filestore_ip': filestore_ip,
            'filestore_name': filestore_name,
            'private_zone_name': private_zone_name,
            'region': region,
            'subnet_name': subnet_name,
            'vpc_name': vpc_name,
            'vpc_peering_name': vpc_peering_name,
            'zones': zones
        }

In [19]:
f = open('res_info.json', 'w')
json.dump(res_info, f)
f.close()
!cat res_info.json

{"vender": "gcp", "cloud_sql_ip": "10.253.0.3", "cloud_sql_name": "topse-wordpress-mysql-20200215112636", "cluster_name": "topse-cluster", "dns_name": "topse.private.", "filestore_ip": "10.196.216.218", "filestore_name": "nfs-server", "private_zone_name": "topse-private-zone", "region": "us-west1", "subnet_name": "topse-subnet", "vpc_name": "topse-vpc", "vpc_peering_name": "topse-vpc-peering", "zones": ["us-west1-a", "us-west1-b"]}

In [20]:
!date

Sat Feb 15 11:42:31 UTC 2020
